In [7]:
#Get the Pacers team_id
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

lakers = [team for team in nba_teams if team['abbreviation'] == 'LAL'][0]
lakers_id = lakers['id']
print(f'lakers_id: {lakers_id}')

pacers_id: 1610612754
lakers_id: 1610612747


In [8]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=lakers_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

Searching through 67 game(s) for the game_id of 0021800848 where LAL @ ATL


In [9]:
# Query for the play by play of that most recent regular season game
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head() #just looking at the head of the data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0021800848,2,12,0,1,7:41 PM,12:00,None,None,None,None,None
1,0021800848,4,10,0,1,7:42 PM,12:00,Jump Ball Dedmon vs. McGee: Tip to Ingram,None,None,None,None
2,0021800848,7,2,98,1,7:42 PM,11:48,None,None,MISS Kuzma 1' Cutting Layup Shot,None,None
3,0021800848,8,4,0,1,7:42 PM,11:45,Young REBOUND (Off:0 Def:1),None,None,None,None
4,0021800848,9,2,101,1,7:42 PM,11:36,MISS Young 12' Driving Floating Jump Shot,None,None,None,None


In [10]:
#Since the datset is fairly large you'll see plenty of elipses(...). 
#If that's the case, you can set the following options to expand the data 
#You can adjust these as you'd like
import pandas
pandas.set_option('display.max_colwidth',250)
pandas.set_option('display.max_rows',250)

In [11]:
#List unique values in the df['EVENTMSGTYPE'] colum
print(f'EVENTMSGTYPE: {sorted(df.EVENTMSGTYPE.unique())}')

EVENTMSGTYPE: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 18]


In [12]:
#For quick refernce, here's an Enum for `EVENTMSGTYPE`
#This list may be incomplete as a thourogh play by play scan is necessary

from enum import Enum

class EventMsgType(Enum):
    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROWfree_throw_attempt = 3
    REBOUND = 4
    TURNOVER = 5
    FOUL = 6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [13]:
#pull the data for a specfic EVENTMSGTYPE
df.loc[df['EVENTMSGTYPE'] == 2].head() #hint: use the EVENTMSGTYPE values above to see different data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
2,0021800848,7,2,98,1,7:42 PM,11:48,None,None,MISS Kuzma 1' Cutting Layup Shot,None,None
4,0021800848,9,2,101,1,7:42 PM,11:36,MISS Young 12' Driving Floating Jump Shot,None,None,None,None
12,0021800848,21,2,1,1,7:44 PM,10:01,MISS Collins 10' Jump Shot,None,None,None,None
17,0021800848,28,2,101,1,7:45 PM,9:29,MISS Young 14' Driving Floating Jump Shot,None,None,None,None
20,0021800848,32,2,1,1,7:45 PM,9:19,None,None,MISS Bullock 26' 3PT Jump Shot,None,None


In [14]:
#List unique values in the df['EVENTMSGTYPE'] column
emt_df = df.loc[df['EVENTMSGTYPE'] == 1]
print(f'EVENTMSGACTIONTYPE: {sorted(emt_df.EVENTMSGACTIONTYPE.unique())}')

EVENTMSGACTIONTYPE: [1, 3, 5, 6, 7, 9, 41, 43, 44, 47, 73, 78, 79, 97, 98, 101, 108]


In [15]:
#pull the data for a specfic EVENTMSGTYPE
df.loc[df['EVENTMSGTYPE'] == 8].head() #hint: use the EVENTMSGTYPE values above to see different data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
47,0021800848,69,8,0,1,7:53 PM,6:27,SUB: Len FOR Dedmon,None,None,None,None
79,0021800848,108,8,0,1,8:00 PM,3:43,SUB: Carter FOR Huerter,None,None,None,None
80,0021800848,109,8,0,1,8:00 PM,3:43,SUB: Bembry FOR Prince,None,None,None,None
81,0021800848,110,8,0,1,8:00 PM,3:43,SUB: Spellman FOR Collins,None,None,None,None
82,0021800848,111,8,0,1,8:00 PM,3:43,None,None,SUB: Stephenson FOR Kuzma,None,None


In [16]:
sub = df.loc[df['EVENTMSGTYPE'] == 8]
sub.head(20)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
47,0021800848,69,8,0,1,7:53 PM,6:27,SUB: Len FOR Dedmon,None,None,None,None
79,0021800848,108,8,0,1,8:00 PM,3:43,SUB: Carter FOR Huerter,None,None,None,None
80,0021800848,109,8,0,1,8:00 PM,3:43,SUB: Bembry FOR Prince,None,None,None,None
81,0021800848,110,8,0,1,8:00 PM,3:43,SUB: Spellman FOR Collins,None,None,None,None
82,0021800848,111,8,0,1,8:00 PM,3:43,None,None,SUB: Stephenson FOR Kuzma,None,None
83,0021800848,112,8,0,1,8:00 PM,3:43,None,None,SUB: Caldwell-Pope FOR McGee,None,None
84,0021800848,113,8,0,1,8:00 PM,3:43,None,None,SUB: Wagner FOR Ingram,None,None
85,0021800848,114,8,0,1,8:00 PM,3:43,None,None,SUB: Rondo FOR Bullock,None,None
94,0021800848,133,8,0,1,8:02 PM,2:15,SUB: Bazemore FOR Young,None,None,None,None
105,0021800848,149,8,0,1,8:04 PM,0:42,SUB: Dedmon FOR Len,None,None,None,None


In [17]:
len(df.columns)

12

In [18]:
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id)
pbp = pbp.get_data_frames()[0]
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,0021800848,2,12,0,1,7:41 PM,12:00,None,None,None,...,None,None,None,0,0,None,NaN,None,None,None
1,0021800848,4,10,0,1,7:42 PM,12:00,Jump Ball Dedmon vs. McGee: Tip to Ingram,None,None,...,Los Angeles,Lakers,LAL,5,1627742,Brandon Ingram,1.610613e+09,Los Angeles,Lakers,LAL
2,0021800848,7,2,98,1,7:42 PM,11:48,None,None,MISS Kuzma 1' Cutting Layup Shot,...,None,None,None,0,0,None,NaN,None,None,None
3,0021800848,8,4,0,1,7:42 PM,11:45,Young REBOUND (Off:0 Def:1),None,None,...,None,None,None,0,0,None,NaN,None,None,None
4,0021800848,9,2,101,1,7:42 PM,11:36,MISS Young 12' Driving Floating Jump Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None


In [19]:
print(list(pbp.columns))

['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD', 'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION', 'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY', 'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY', 'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY', 'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION']


In [25]:
#Get all Laker Games
#Get all game dates
#Get all lineups for each game date by date

In [49]:
#Get this season's lakers games
#print(games)
print(type(games))
print(type(games[0]))
print(list(games[0]))
print(games[0]["GAME_DATE"])
print(len(games))

#Get all the game dates
gamedates = []
for game in games: 
    gamedates.append(game["GAME_DATE"])
print(gamedates[:5])

#Combine dataframes
for date in gamedates:
    tempdf = teamdashlineups.TeamDashLineups(
        lakers_id,date_from_nullable = date, date_to_nullable = date).get_data_frames()[1]
    df3.append(tempdf)


<class 'list'>
<class 'dict'>
['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']
2019-02-12
67
['2019-02-12', '2019-02-10', '2019-02-07', '2019-02-05', '2019-02-02']


In [44]:
#Filter lineups by date
from nba_api.stats.endpoints import teamdashlineups
temp = "date_from_" + str(games[0]["GAME_DATE"])
temp2 = "date_to_" + str(games[0]["GAME_DATE"])
print(temp)
#df2 = teamdashlineups.TeamDashLineups(lakers_id,temp,temp2).get_data_frames()[1]
df2 = teamdashlineups.TeamDashLineups(lakers_id,date_from_nullable = "2019-02-11", date_to_nullable = "2019-02-13").get_data_frames()[1]
df2.head()


date_from_2019-02-12


,GROUP_SET,GROUP_ID,GROUP_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-2544-201580-203493-1627742-1628398-,L. James - J. McGee - R. Bullock - B. Ingram - K. Kuzma,1,0,1,0.0,11.316667,11,21,...,1,1,17,9,1,12,16,1,1,17
1,Lineups,-2544-200765-203493-1627742-1628398-,L. James - R. Rondo - R. Bullock - B. Ingram - K. Kuzma,1,0,1,0.0,9.616667,8,18,...,2,1,16,1,6,1,17,3,2,4
2,Lineups,-2544-200765-203484-1627742-1628398-,L. James - R. Rondo - K. Caldwell-Pope - B. Ingram - K. Kuzma,1,0,1,0.0,5.083333,2,9,...,4,10,14,2,2,12,14,2,8,11
3,Lineups,-2544-200765-203484-203493-1628398-,L. James - R. Rondo - K. Caldwell-Pope - R. Bullock - K. Kuzma,1,0,1,0.0,4.733333,4,13,...,3,3,10,9,6,12,14,10,3,15
4,Lineups,-2544-200765-202362-203484-1629021-,L. James - R. Rondo - L. Stephenson - K. Caldwell-Pope - M. Wagner,1,0,1,0.0,3.716667,3,7,...,5,7,14,9,6,1,1,5,4,16


In [34]:
#Present the total and say this is totals
#THen look at per game
#look at the per 100 possessions - which would represent the estimated 
#Need to find how the stats are calculated

In [ ]:
#Lebron - games - scores - win/loss > Do for Kuzma too > 

In [1]:
#Need to work with play by play 

# Lineups 

In [20]:
from nba_api.stats.endpoints import teamdashlineups

In [21]:
df2 = teamdashlineups.TeamDashLineups(lakers_id).get_data_frames()[1]
df2.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-2544-201580-1627742-1628366-1628398-,L. James - J. McGee - B. Ingram - L. Ball - K. Kuzma,18,13,5,0.722,233.900000,216,433,...,1,1,544,1,1,544,544,1,1,339
1,Lineups,-2544-201580-1628366-1628398-1628404-,L. James - J. McGee - L. Ball - K. Kuzma - J. Hart,15,8,7,0.533,133.493333,131,258,...,2,2,543,2,2,543,543,2,2,1
2,Lineups,-201580-1627742-1628366-1628398-1628404-,J. McGee - B. Ingram - L. Ball - K. Kuzma - J. Hart,9,4,5,0.444,82.505000,71,159,...,3,3,542,3,3,534,542,6,3,544
3,Lineups,-2199-2544-1628366-1628398-1628404-,T. Chandler - L. James - L. Ball - K. Kuzma - J. Hart,13,7,6,0.538,65.525000,60,119,...,5,6,540,4,6,541,540,3,4,138
4,Lineups,-2199-2544-1627742-1628366-1628398-,T. Chandler - L. James - B. Ingram - L. Ball - K. Kuzma,15,12,3,0.800,61.270000,56,116,...,4,5,541,10,3,542,541,5,5,24


In [22]:
print(list(df2.columns))

['GROUP_SET', 'GROUP_ID', 'GROUP_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK']


In [23]:
df2.sort_values('GP', ascending = False).head()

,GROUP_SET,GROUP_ID,GROUP_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-2544-201580-1627742-1628366-1628398-,L. James - J. McGee - B. Ingram - L. Ball - K. Kuzma,18,13,5,0.722,233.900000,216,433,...,1,1,544,1,1,544,544,1,1,339
4,Lineups,-2199-2544-1627742-1628366-1628398-,T. Chandler - L. James - B. Ingram - L. Ball - K. Kuzma,15,12,3,0.800,61.270000,56,116,...,4,5,541,10,3,542,541,5,5,24
1,Lineups,-2544-201580-1628366-1628398-1628404-,L. James - J. McGee - L. Ball - K. Kuzma - J. Hart,15,8,7,0.533,133.493333,131,258,...,2,2,543,2,2,543,543,2,2,1
3,Lineups,-2199-2544-1628366-1628398-1628404-,T. Chandler - L. James - L. Ball - K. Kuzma - J. Hart,13,7,6,0.538,65.525000,60,119,...,5,6,540,4,6,541,540,3,4,138
8,Lineups,-2199-2544-203484-1628366-1628398-,T. Chandler - L. James - K. Caldwell-Pope - L. Ball - K. Kuzma,13,7,6,0.538,38.866667,35,70,...,12,7,524,5,17,534,525,9,9,7


In [24]:
df2.sort_values('FGM', ascending = False).head()

,GROUP_SET,GROUP_ID,GROUP_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-2544-201580-1627742-1628366-1628398-,L. James - J. McGee - B. Ingram - L. Ball - K. Kuzma,18,13,5,0.722,233.900000,216,433,...,1,1,544,1,1,544,544,1,1,339
1,Lineups,-2544-201580-1628366-1628398-1628404-,L. James - J. McGee - L. Ball - K. Kuzma - J. Hart,15,8,7,0.533,133.493333,131,258,...,2,2,543,2,2,543,543,2,2,1
2,Lineups,-201580-1627742-1628366-1628398-1628404-,J. McGee - B. Ingram - L. Ball - K. Kuzma - J. Hart,9,4,5,0.444,82.505000,71,159,...,3,3,542,3,3,534,542,6,3,544
3,Lineups,-2199-2544-1628366-1628398-1628404-,T. Chandler - L. James - L. Ball - K. Kuzma - J. Hart,13,7,6,0.538,65.525000,60,119,...,5,6,540,4,6,541,540,3,4,138
4,Lineups,-2199-2544-1627742-1628366-1628398-,T. Chandler - L. James - B. Ingram - L. Ball - K. Kuzma,15,12,3,0.800,61.270000,56,116,...,4,5,541,10,3,542,541,5,5,24
